# Document Chracterizzation


## Import of relevant libraries 

In [1]:
# Import of all the libraries used for the project
import os
import shutil
import numpy as np
import pandas as pd
import requests
from bs4 import BeautifulSoup
import re
from operator import itemgetter

## Dataset import

In [2]:
# Defined a simple scraper  to go through all the link, visit the page and copy all the txt in the paragraph tag of the HTML file

def scraper(input):
    # create an empty array
    content = []

    # for loop on all the links in the dataframe
    for i in range(len(input["URL"])):
        url = input["URL"][i]
        # text = url
        text = ' '

        # check if actually the url is correct orr not
        if not (url.startswith('https://') or url.startswith('http://')):
            text += ' Link not good'
            content.append(text) 
            continue
        
        # Request the url and copy of the HTMl 
        page = requests.get(url)
        
        # Check if the response is positive or not, if not set that page was not found 
        if page.status_code != 200:
            text += ' Page not able to find'

        # Else take the text from all the p tag of the HTML content  
        else:
            soup = BeautifulSoup(page.content, 'html.parser')
            for p in soup.find_all('p'):
                text = text + ' ' + p.text
        
        content.append(text) 
    return content

In [4]:
## import the CSV file with the labels

data_scraper = pd.read_csv('Document Characterisation Data.csv', sep=',')
#print(len(data_scraper["URL"]))

# Calling the scraper function
data = scraper(data_scraper)

# Adding the scraped data into the Dataframe  
data_scraper['CONTENT'] = data
data_scraper

,URL,CLASS,CONTENT
0,https://issafrica.org/iss-today/time-to-tackle...,Analytical Report,"2020-11-25 On 26 September, hundreds of Mali..."
1,http://www.understandingwar.org/backgrounder/r...,Analytical Report,"By George Barros November 16, 2020 The Kreml..."
2,http://www.understandingwar.org/backgrounder/r...,Analytical Report,"By Ezgi Yazici November 13, 2020 Key Takeawa..."
3,http://www.understandingwar.org/backgrounder/w...,Analytical Report,By Katherine Lawlor with Brandon Wallace Key...
4,http://www.understandingwar.org/backgrounder/r...,Analytical Report,By Isabel Ivanescu and Ezgi Yazici Key Takea...
...,...,...,...
232,https://www.journalism.org/2019/03/26/for-loca...,Analytical Report,"Numbers, Facts and Trends Shaping Your World..."
233,https://www.journalism.org/2017/10/02/covering...,Analytical Report,"Numbers, Facts and Trends Shaping Your World..."
234,https://ctc.usma.edu/afghanistans-future-emira...,Analytical Report,Main Menu Seth G. Jones \nShare via:\n\n\n\n...
235,https://ctc.usma.edu/back-into-the-shadows-the...,Analytical Report,Main Menu Michael Knights \nShare via:\n\n\n...


In [5]:
# Function to load the files from local drive into Pandas data frame

def readFiles (src, fileClass):
    src_files = os.listdir(src)
    fileType = []
    content = []
    for i in range(len(src_files)):
        fileType.append(fileClass)
        glove_file = open(src + src_files[i], "r", encoding="utf8")
        glove_vectors_list = [word_and_vector.strip() for word_and_vector in glove_file.readlines()]
        content.append(" ".join(glove_vectors_list))
    
    columns = ["URL", "CLASS", "CONTENT"]
    df = pd.DataFrame(columns=columns)

    df["URL"] = src_files
    df["CLASS"] = fileType
    df["CONTENT"] = content

    return df


In [8]:
src = "./Data/Analytical Report/"
df_Analytical_Report = readFiles(src, "Analytical Report")

src = "./Data/Analytical Report2/"
df_Analytical_Report2 = readFiles(src, "Analytical Report")

src = "./Data/Incident Report/"
df_Incident_Report = readFiles(src, "Incident Report")

src = "./Data/Situation Report/"
df_Situation_Report = readFiles(src, "Situation Report")

toConcat = [data_scraper, df_Analytical_Report, df_Analytical_Report2, df_Incident_Report, df_Situation_Report]

data_input = pd.concat(toConcat)
data_input

,URL,CLASS,CONTENT
0,https://issafrica.org/iss-today/time-to-tackle...,Analytical Report,"2020-11-25 On 26 September, hundreds of Mali..."
1,http://www.understandingwar.org/backgrounder/r...,Analytical Report,"By George Barros November 16, 2020 The Kreml..."
2,http://www.understandingwar.org/backgrounder/r...,Analytical Report,"By Ezgi Yazici November 13, 2020 Key Takeawa..."
3,http://www.understandingwar.org/backgrounder/w...,Analytical Report,By Katherine Lawlor with Brandon Wallace Key...
4,http://www.understandingwar.org/backgrounder/r...,Analytical Report,By Isabel Ivanescu and Ezgi Yazici Key Takea...
...,...,...,...
65,int-rep-o-18-e.txt,Situation Report,Deputies are investigating after an explosive ...
66,int-rep-o-18-f.txt,Situation Report,Residents at a Springfield apartment complex w...
67,int-rep-o-18-g.txt,Situation Report,Several handheld explosive devices were found ...
68,int-rep-o-18-h.txt,Situation Report,Boulder police taped off the U.S. Bank at Broa...


## Experimental Data Analysis 

### Data frame informations and duplicates Duplicates

In [9]:
data_input.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 418 entries, 0 to 69
Data columns (total 3 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   URL      418 non-null    object
 1   CLASS    418 non-null    object
 2   CONTENT  418 non-null    object
dtypes: object(3)
memory usage: 13.1+ KB


In [14]:
# Number of items per class
print("Number of Analytical Report ", len(data_input[data_input['CLASS'] == 'Analytical Report']))
print("Number of Situation Report ", len(data_input[data_input['CLASS'] == 'Situation Report']))
print("Number of Incident Report ", len(data_input[data_input['CLASS'] == 'Incident Report']))
print("Number of Profile Report ", len(data_input[data_input['CLASS'] == 'Profile Report']))


Number of Analytical Report  131
Number of Situation Report  99
Number of Incident Report  88
Number of Profile Report  100


In [25]:
# Dropping the duplicate items 
data_input.drop_duplicates( inplace=True)


In [26]:
# Number of items per class after dropping the duplicates
print("Number of Analytical Report ", len(data_input[data_input['CLASS'] == 'Analytical Report']))
print("Number of Situation Report ", len(data_input[data_input['CLASS'] == 'Situation Report']))
print("Number of Incident Report ", len(data_input[data_input['CLASS'] == 'Incident Report']))
print("Number of Profile Report ", len(data_input[data_input['CLASS'] == 'Profile Report']))


Number of Analytical Report  92
Number of Situation Report  98
Number of Incident Report  88
Number of Profile Report  98


In [27]:
data_input.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 376 entries, 0 to 69
Data columns (total 3 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   URL      376 non-null    object
 1   CLASS    376 non-null    object
 2   CONTENT  376 non-null    object
dtypes: object(3)
memory usage: 11.8+ KB


### Pre Processing the data

In [76]:
# tockenizzation of the corpus 
def tokenize(corpus):

    # The backslash is an escape character to let the interpreter know we mean to use it as a string literal (- or ') 
    re_punctuation_string = '[\s,/.?\-\'`’]'  # split on spaces (\s),  commas (,), slash (/), fullstop (.), question marks (?), hyphens (-), and apostrophe (').
    tokenized = re.split(re_punctuation_string, corpus) # in python's regex, [...] is an alternative to writing .|.|.
    tokenized = list(filter(None, tokenized)) # remove empty strings from list 
    tokenized = [token.lower() for token in tokenized] # Lowercasing everything
    tokenized = ['<Number>' if token.isdigit() else token for token in tokenized ]  # converting all the numbers to <Number>


    return tokenized    

corpus = data_input['Content'][0]        
tokenized_corpus = tokenize(corpus)
print(tokenized_corpus)

['<Number>', '<Number>', '<Number>', 'on', '<Number>', 'september', 'hundreds', 'of', 'malian', 'women', 'and', 'some', 'men', 'rallied', 'in', 'bamako', 'to', 'protest', 'against', 'the', 'trivialisation', 'of', 'violence', 'against', 'women', 'this', 'was', 'prompted', 'by', 'the', 'revelation', 'on', 'social', 'media', 'of', 'alleged', 'intimate', 'partner', 'abuse', 'involving', 'a', 'famous', 'malian', 'artist', 'the', 'story', 'broke', 'just', 'two', 'months', 'before', 'the', '<Number>', 'november', 'annual', 'international', 'day', 'for', 'the', 'elimination', 'of', 'violence', 'against', 'women', 'the', 'case', 'now', 'pending', 'in', 'court', 'sheds', 'light', 'on', 'a', 'society', 'wide', 'problem', 'that', 's', 'often', 'silenced', 'by', 'deeply', 'entrenched', 'social', 'norms', 'and', 'religious', 'discourse', 'fear', 'on', 'the', 'part', 'of', 'the', 'political', 'elite', 'to', 'confront', 'the', 'divisive', 'issue', 'makes', 'matters', 'worse', 'official', 'figures', 'o

In [77]:
# function to count the tockenization
def distinct_tokens_count(tokenized_corpus):

    distinct_tokens_count = {}
    for t_review in tokenized_corpus:

        if t_review not in distinct_tokens_count.keys():
            distinct_tokens_count[t_review] = 1
        else:
            distinct_tokens_count[t_review] += 1

    # Sort the key value to greter to smaller
    distinct_tokens_count = sorted(distinct_tokens_count.items(), key=itemgetter(1), reverse=True) 

    for token, count in distinct_tokens_count:
        print("{:<14s} {:<10d}".format(token, count))
    return distinct_tokens_count

a = distinct_tokens_count(tokenized_corpus)

the            127       
and            83        
women          82        
to             70        
of             69        
in             62        
a              44        
s              39        
violence       36        
against        30        
<Number>       29        
on             28        
for            28        
this           26        
by             26        
be             22        
law            18        
that           16        
rights         16        
will           14        
transition     12        
was            10        
mali           10        
but            10        
gender         10        
national       10        
transitional   10        
t              10        
all            10        
an             10        
agenda         10        
need           10        
also           9         
social         8         
society        8         
religious      8         
political      8         
are            8         
country     

{'one': 5, 'three': 2, 'five': 1, 'two': 10, 'four': 8}
[('two', 10), ('four', 8), ('one', 5), ('three', 2), ('five', 1)]


Pre trained BERT models

https://pypi.org/project/pytorch-pretrained-bert/

https://pytorch.org/hub/huggingface_pytorch-transformers/

https://medium.com/@aniruddha.choudhury94/part-2-bert-fine-tuning-tutorial-with-pytorch-for-text-classification-on-the-corpus-of-linguistic-18057ce330e1
